In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

Dados retirados do MovieLens, Link: https://grouplens.org/datasets/movielens/

In [ ]:
url_movies = '/content/drive/MyDrive/Colab Notebooks/Data/movies.csv'
url_ratings = '/content/drive/MyDrive/Colab Notebooks/Data/ratings.csv'
#url_tags = '/content/drive/MyDrive/Colab Notebooks/Data/tags.csv'
#url_genome_score = '/content/drive/MyDrive/Colab Notebooks/Data/genome-scores.csv'
#url_genome_tags = '/content/drive/MyDrive/Colab Notebooks/Data/genome-tags.csv'
url_links = '/content/drive/MyDrive/Colab Notebooks/Data/links.csv'

df_links = pd.read_csv(url_links)
df_movies = pd.read_csv(url_movies)
df_ratings = pd.read_csv(url_ratings)
#df_tags = pd.read_csv(url_tags)
#df_genome_score = pd.read_csv(url_genome_score)
#df_genome_tags = pd.read_csv(url_genome_tags)

Os datasets de Tags, genome score e genome tags, foram testados porém não apresentaram um desempenho bom com a inteligência artificial, então foram descartados

In [ ]:
display(df_links, df_movies, df_ratings)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
62418,209157,6671244,499546.0
62419,209159,297986,63407.0
62420,209163,6755366,553036.0
62421,209169,249603,162892.0


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [ ]:
df_movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [ ]:
df_ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
df_ratings.nunique()

userId         162541
movieId         59047
rating             10
timestamp    20115267
dtype: int64

In [ ]:
df_movies.nunique()

movieId    62423
title      62325
genres      1639
dtype: int64

In [ ]:
#df_genome_score.groupby('movieId').value_counts()

In [ ]:
#display(df_movies.head(),df_ratings.head())

In [ ]:
df_movies.nunique()

movieId    62423
title      62325
genres      1639
dtype: int64

In [ ]:
df_ratings_media = pd.DataFrame(df_ratings.drop(['timestamp','userId'],axis = 1).groupby('movieId').mean())
df_ratings_media

,rating
movieId,
1,3.893708
2,3.251527
3,3.142028
4,2.853547
5,3.058434
...,...
209157,1.500000
209159,3.000000
209163,4.500000


In [ ]:
df = df_movies.join(df_ratings_media, on = 'movieId')

In [ ]:
#df_tags = df_tags.drop(['timestamp','userId'],axis = 1).set_index('movieId')

In [ ]:
#df = df.join(df_tags)

In [ ]:
#df_genome_score_tag = df_genome_score.join(df_genome_tags.set_index('tagId'),on='tagId').drop('tagId', axis = 1)

In [ ]:
#df_genome_score_tag[df_genome_score_tag['movieId']==1]

In [ ]:
#df = pd.merge(df,df_genome_score_tag,right_on = 'movieId',left_on='movieId').reset_index().drop(['index','relevance','tag'], axis = 1).drop_duplicates().reset_index().drop('index',axis = 1)

In [ ]:
df.isnull().sum()

movieId       0
title         0
genres        0
rating     3376
dtype: int64

In [ ]:
#df = df[df.index<3500]

In [ ]:
x = pd.merge(df['movieId'],pd.DataFrame(df['genres'].str.split('|').explode()),left_index=True,right_index=True).set_index('movieId')

df_movies2 = pd.merge(df.drop('genres',axis=1),x,left_on = 'movieId',right_index = True).reset_index().drop('index',axis=1)


dados_filmes_dummies = pd.merge(df.drop('genres',axis = 1),x,right_index=True,left_on='movieId').reset_index().drop('index', axis = 1)

In [ ]:
df_movies2.nunique()

movieId    62423
title      62325
rating     12972
genres        20
dtype: int64

In [ ]:
df_movies2.shape

(112307, 4)

In [ ]:
del df_movies
del df_ratings


In [ ]:

ohe = OneHotEncoder(dtype=int)
colunas_ohe = ohe.fit_transform(x).toarray()
df_movies2 = df_movies2.drop(['genres'], axis=1)

dados_filmes_dummies = pd.concat([df_movies2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['genres']))], axis=1).reset_index().drop('index', axis = 1).fillna(0).groupby(['movieId','rating','title'],as_index=False).sum()
dados_filmes_dummies.head()


,movieId,rating,title,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western
0,1,3.893708,Toy Story (1995),0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,3.251527,Jumanji (1995),0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3.142028,Grumpier Old Men (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,2.853547,Waiting to Exhale (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,3.058434,Father of the Bride Part II (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dados_filmes_dummies.tail()

,movieId,rating,title,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western
62418,209157,1.5,We (2018),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62419,209159,3.0,Window of the Soul (2001),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62420,209163,4.5,Bad Poems (2018),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
62421,209169,3.0,A Girl Thing (2001),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62422,209171,3.0,Women of Devil's Island (1962),0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
pd.concat([df_movies2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['genres']))], axis=1).fillna(0).groupby(['movieId','rating','title'],as_index=False).sum()

,movieId,rating,title,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western
0,1,3.893708,Toy Story (1995),0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,3.251527,Jumanji (1995),0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3.142028,Grumpier Old Men (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,2.853547,Waiting to Exhale (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,3.058434,Father of the Bride Part II (1995),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,209157,1.500000,We (2018),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62419,209159,3.000000,Window of the Soul (2001),0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62420,209163,4.500000,Bad Poems (2018),0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
62421,209169,3.000000,A Girl Thing (2001),1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dados_filmes_dummies[dados_filmes_dummies['rating'].isnull()]

,movieId,rating,title,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western


In [ ]:
df_movies2.shape

(112307, 3)

In [ ]:
colunas_ohe.shape

(112307, 20)

In [ ]:
x.shape

(112307, 1)

In [ ]:
dados_filmes_dummies.shape

(62423, 23)

In [ ]:
df.shape

(62423, 4)

O código comentado foram as partes de tratamento de dados dos datasets reprovados

In [ ]:
#dados_filmes_dummies2 = pd.merge(df.drop('genres',axis = 1),df_genome_score_tag,right_on = 'movieId',left_on='movieId').reset_index().drop('index', axis = 1)
#del df_genome_score_tag

In [ ]:
#dados_filmes_dummies2 = dados_filmes_dummies2[dados_filmes_dummies2['relevance']>0.5].reset_index().drop('index', axis = 1)

In [ ]:
#x = pd.DataFrame(dados_filmes_dummies2['tag'])
#colunas_ohe = ohe.fit_transform(x).toarray()



In [ ]:
#colunas_ohe.shape

In [ ]:
#colunas_ohe = [dados_filmes_dummies2['relevance'][i]*colunas_ohe[i] for i in dados_filmes_dummies2.index]

In [ ]:
#dados_filmes_dummies2 = dados_filmes_dummies2.drop(['tag','relevance'], axis=1)

#dados_filmes_dummies2 = pd.concat([dados_filmes_dummies2, pd.DataFrame(colunas_ohe, columns=ohe.get_feature_names_out(['tag']))], axis=1).reset_index().drop('index', axis = 1).groupby(['movieId','rating'],as_index=False).sum()

#dados_filmes_dummies2.head()

In [ ]:
#dados_filmes_dummies2

In [ ]:
#dados_filmes_dummies = pd.merge(dados_filmes_dummies,dados_filmes_dummies2,right_on='movieId',left_on='movieId').reset_index().drop('index', axis = 1)

In [ ]:
#dados_filmes_dummies

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
#dados_filmes_dummies = dados_filmes_dummies.drop(['rating_y','rating_x'],axis = 1)

In [ ]:
dados_filmes_dummies[dados_filmes_dummies['title'] == 'Poltergeist (1982)']


,movieId,rating,title,genres_(no genres listed),genres_Action,genres_Adventure,genres_Animation,genres_Children,genres_Comedy,genres_Crime,...,genres_Film-Noir,genres_Horror,genres_IMAX,genres_Musical,genres_Mystery,genres_Romance,genres_Sci-Fi,genres_Thriller,genres_War,genres_Western
1905,1994,3.52212,Poltergeist (1982),0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [ ]:
SEED = 9999
np.random.seed(9999)

pca_pipeline = Pipeline([('scaler', StandardScaler()), ('PCA', PCA(n_components=2, random_state=SEED))])

In [ ]:

film_embedding_pca = pca_pipeline.fit_transform(dados_filmes_dummies.drop(['movieId','title'], axis=1))
projection_m = pd.DataFrame(data=film_embedding_pca)


In [ ]:
from sklearn.cluster import KMeans

In [ ]:
projection_m

,0,1
0,6.466402,4.415020
1,4.176512,3.572382
2,0.915164,-1.172247
3,0.453852,-1.489229
4,0.706601,-0.634341
...,...,...
62418,-0.401867,-0.565331
62419,0.354478,-1.084435
62420,0.325034,-1.192599
62421,0.269747,-1.013102


In [ ]:
kmeans_pca_pipeline = KMeans(n_clusters=100, verbose=False, random_state=SEED)

kmeans_pca_pipeline.fit(projection_m)

df['cluster_pca'] = kmeans_pca_pipeline.predict(projection_m)
projection_m['cluster_pca'] = kmeans_pca_pipeline.predict(projection_m)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
projection_m['title'] = df['title']

In [ ]:
projection_m

,0,1,cluster_pca,title
0,6.466402,4.415020,21,Toy Story (1995)
1,4.176512,3.572382,82,Jumanji (1995)
2,0.915164,-1.172247,77,Grumpier Old Men (1995)
3,0.453852,-1.489229,50,Waiting to Exhale (1995)
4,0.706601,-0.634341,1,Father of the Bride Part II (1995)
...,...,...,...,...
62418,-0.401867,-0.565331,7,We (2018)
62419,0.354478,-1.084435,72,Window of the Soul (2001)
62420,0.325034,-1.192599,36,Bad Poems (2018)
62421,0.269747,-1.013102,9,A Girl Thing (2001)


In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(
   projection_m, x=0, y=1, color='cluster_pca', hover_data=[0, 1, 'title'])
fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
nome_filme = 'Toy Story (1995)'

In [ ]:
from pandas.core.dtypes.cast import maybe_upcast
from sklearn.metrics.pairwise import euclidean_distances

cluster = list(projection_m[projection_m['title']== nome_filme]['cluster_pca'])[0]
filmes_recomendadas = projection_m[projection_m['cluster_pca']== cluster][[0, 1, 'title']]
x_filme = list(projection_m[projection_m['title']== nome_filme][0])[0]
y_filme = list(projection_m[projection_m['title']== nome_filme][1])[0]

#distâncias euclidianas
distancias = euclidean_distances(filmes_recomendadas[[0, 1]], [[x_filme, y_filme]])
filmes_recomendadas['movieId'] = df['movieId']
filmes_recomendadas['distancias']= distancias
recomendada = filmes_recomendadas.sort_values('distancias').head(10)
recomendada

,0,1,title,movieId,distancias
0,6.466402,4.415020,Toy Story (1995),1,0.000000
43614,6.464383,4.421130,Moana (2016),166461,0.006435
4780,6.464324,4.421308,"Monsters, Inc. (2001)",4886,0.006623
3021,6.462611,4.426490,Toy Story 2 (1999),3114,0.012080
52826,6.471301,4.400197,Tangled: Before Ever After (2017),186159,0.015612
55898,6.460664,4.432380,Penguin Highway (2018),192981,0.018283
23212,6.459778,4.435062,The Magic Crystal (2011),117454,0.021108
3912,6.453830,4.453059,"Emperor's New Groove, The (2000)",4016,0.040063
9949,6.443072,4.485607,DuckTales: The Movie - Treasure of the Lost La...,33463,0.074342
22353,6.440632,4.492991,"Boxtrolls, The (2014)",114552,0.082119
